In [1]:
import math
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def relu(x):
    return max(0, x)

def relu_derivative(x):
    return 1 if x > 0 else 0

class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_rate = dropout_rate

        self.weights1 = [[random.uniform(-1, 1) for _ in range(hidden_size)] for _ in range(input_size)]
        self.bias1 = [random.uniform(-1, 1) for _ in range(hidden_size)]

        self.weights2 = [[random.uniform(-1, 1) for _ in range(hidden_size)] for _ in range(hidden_size)]
        self.bias2 = [random.uniform(-1, 1) for _ in range(hidden_size)]

        self.weights3 = [[random.uniform(-1, 1) for _ in range(output_size)] for _ in range(hidden_size)]
        self.bias3 = [random.uniform(-1, 1) for _ in range(output_size)]

    def apply_dropout(self, layer):
        return [neuron * (random.random() > self.dropout_rate) for neuron in layer]

    def feedforward(self, x, training=False):
        self.hidden1 = [relu(sum(x[j] * self.weights1[j][i] for j in range(self.input_size)) + self.bias1[i])
                        for i in range(self.hidden_size)]
        self.hidden2 = [relu(sum(self.hidden1[j] * self.weights2[j][i] for j in range(self.hidden_size)) + self.bias2[i])
                        for i in range(self.hidden_size)]
        if training:
            self.hidden2 = self.apply_dropout(self.hidden2)
        self.output = [sigmoid(sum(self.hidden2[j] * self.weights3[j][i] for j in range(self.hidden_size)) + self.bias3[i])
                       for i in range(self.output_size)]
        return self.output[0]

    def backpropagate(self, x, y, learning_rate):
        output_error = (self.output[0] - y) * sigmoid_derivative(self.output[0])

        d_weights3 = [[output_error * self.hidden2[j] for j in range(self.hidden_size)]]
        d_bias3 = [output_error]

        hidden2_error = [output_error * self.weights3[j][0] * relu_derivative(self.hidden2[j]) for j in range(self.hidden_size)]

        d_weights2 = [[hidden2_error[i] * self.hidden1[j] for j in range(self.hidden_size)] for i in range(self.hidden_size)]
        d_bias2 = hidden2_error

        hidden1_error = [sum(hidden2_error[k] * self.weights2[i][k] for k in range(self.hidden_size)) *
                         relu_derivative(self.hidden1[i]) for i in range(self.hidden_size)]

        d_weights1 = [[hidden1_error[i] * x[j] for j in range(self.input_size)] for i in range(self.hidden_size)]
        d_bias1 = hidden1_error

        for i in range(self.hidden_size):
            for j in range(self.output_size):
                self.weights3[i][j] -= learning_rate * d_weights3[0][i]
            self.bias3[j] -= learning_rate * d_bias3[j]

        for i in range(self.hidden_size):
            for j in range(self.hidden_size):
                self.weights2[i][j] -= learning_rate * d_weights2[i][j]
            self.bias2[i] -= learning_rate * d_bias2[i]

        for i in range(self.hidden_size):
            for j in range(self.input_size):
                self.weights1[j][i] -= learning_rate * d_weights1[i][j]
            self.bias1[i] -= learning_rate * d_bias1[i]

    def train(self, data, labels, learning_rate, stop_probability=0.01):
        epoch = 0
        while True:
            for x, y in zip(data, labels):
                self.feedforward(x, training=True)
                self.backpropagate(x, y, learning_rate)
            epoch += 1
            if random.random() < stop_probability:
                print(f"Training stopped after {epoch} epochs.")
                break

def pearson_arrow_logic(x):
    return (not x[0] or x[1]) and (not x[2] or x[3]) and (not x[4] or True)

input_size = int(input("Enter the input size: "))
hidden_size = int(input("Enter the hidden layer size: "))
output_size = int(input("Enter the output size: "))
dropout_rate = float(input("Enter the dropout rate (0-1): "))
learning_rate = float(input("Enter the learning rate: "))

data = [[random.randint(0, 1) for _ in range(input_size)] for _ in range(1000)]
labels = [pearson_arrow_logic(x) for x in data]

train_size = int(0.8 * len(data))
train_data = data[:train_size]
train_labels = labels[:train_size]
test_data = data[train_size:]
test_labels = labels[train_size:]

nn = SimpleNeuralNetwork(input_size, hidden_size, output_size, dropout_rate)
nn.train(train_data, train_labels, learning_rate)

predictions = [round(nn.feedforward(x)) for x in test_data]

accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

data = [
    [0, 0, 0, 0, 0], 
    [0, 1, 1, 1, 1], 
    [1, 0, 1, 0, 0], 
    [1, 1, 1, 1, 0]   
]
for x in data:
    print(f"Input: {x}, Predicted Output: {bool(round(nn.feedforward(x)))}, True Output: {bool(pearson_arrow_logic(x))}")


KeyboardInterrupt: Interrupted by user